In [10]:
import lxml
import pymysql
from urllib.request import urlopen
import requests
from lxml import etree
import urllib
import html
import time
import random
from urllib.error import URLError
from urllib.request import ProxyHandler,build_opener
import http

In [11]:
# url='https://movie.douban.com/top250?start=0'
# headers={
#     "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'
# }

# html=urllib.request.Request(url,headers=headers)

In [12]:
#s = request.Session()

In [13]:
# 登陆来爬取
def login_douban():
    login_url= 'https://accounts.douban.com/j/mobile/login/basic'
    data={'name':'13549867105',
          'password': 'wu20011210',
          'remember': 'false'}
    headers={'Cookie':'ll="118281"; bid=6k4D4UvewhU; gr_user_id=b057d2a7-2726-4d47-9486-25044db1c016; _pk_ref.100001.2fad=%5B%22%22%2C%22%22%2C1618933620%2C%22https%3A%2F%2Fsec.douban.com%2F%22%5D; _pk_id.100001.2fad=99b2f2328cefa0be.1618933620.1.1618934723.1618933620.; push_doumail_num=0; __utmv=30149280.14442; push_noty_num=0; ap_v=0,6.0; __gads=ID=9a0f946ac5a01053:T=1619002297:S=ALNI_MYjRvk8BUpqPhYL3rbIzPNMiQaJJQ; __utma=30149280.860125068.1618913183.1619002280.1619009283.8; __utmc=30149280; __utmz=30149280.1619009283.8.6.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utmt=1; apiKey=; __utmb=30149280.3.10.1619009283; login_start_time=1619009317128',
           'Referer': 'https://accounts.douban.com/passport/login_popup?login_source=anony',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}
    try:
        res = requests.post(login_url,headers=headers,data=data)
        res.raise_for_status()
    except:
        print("登陆失败")
    print(res.text)
login_douban()
         

{"status":"failed","message":"captcha_required","description":"需要图形验证码","payload":{"tc_app_id":"2044348370","captcha_signature_sample":"2:6,15:c","touch_cap_url":"https:\/\/ssl.captcha.qq.com\/TCaptcha.js","captcha_id":"login:captcha:3wU4NYdI54nauUKedvhSRPRc","captcha_image_url":"https:\/\/accounts.douban.com\/j\/captcha\/show?vid=login:captcha:3wU4NYdI54nauUKedvhSRPRc&size=small"}}


In [14]:
# # 保存会话状态
# cookie= http.cookiejar.CookieJar()
# handler=urllib.request.HTTPCookieProcessor(cookie)
# opener = urllib.request.build_opener(handler)
# opener(url)

In [15]:
conn = pymysql.connect(  # 连接本地数据库
    host="localhost",
    user="root",
    password="9834459687", # 填上自己的密码
    database="test",
    charset="utf8"
)
conn.set_charset('utf8')#设置编码
cursor = conn.cursor()  # 得到一个可以执行SQL语句的光标对象

cursor.execute("DROP TABLE IF EXISTS MovieInfo") # 如果表存在则删除


0

In [16]:
sql = """
    CREATE TABLE MovieInfo (
    `id` INT auto_increment PRIMARY KEY ,
    `MovieRank` VARCHAR(10) ,
    `MovieTitle` VARCHAR(255) ,
    `MovieType` VARCHAR(255)  ,
    `MovieStar` VARCHAR(255) ,
    `MovieQuote` VARCHAR(255),
    `MoiveURL` VARCHAR(255) ,
    `ImageURL` VARCHAR(255) 
    )
"""
try:
    cursor.execute(sql) # 执行创建表操作
except Exception as e:
    print(e)
    # 发生错误则回滚
    conn.rollback()
conn.close()

In [17]:
def crawling(page_num):
    headers={'Cookie':'ll="118281"; bid=6k4D4UvewhU; gr_user_id=b057d2a7-2726-4d47-9486-25044db1c016; _pk_ref.100001.2fad=%5B%22%22%2C%22%22%2C1618933620%2C%22https%3A%2F%2Fsec.douban.com%2F%22%5D; _pk_id.100001.2fad=99b2f2328cefa0be.1618933620.1.1618934723.1618933620.; push_doumail_num=0; __utmv=30149280.14442; push_noty_num=0; ap_v=0,6.0; __gads=ID=9a0f946ac5a01053:T=1619002297:S=ALNI_MYjRvk8BUpqPhYL3rbIzPNMiQaJJQ; __utma=30149280.860125068.1618913183.1619002280.1619009283.8; __utmc=30149280; __utmz=30149280.1619009283.8.6.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utmt=1; apiKey=; __utmb=30149280.3.10.1619009283; login_start_time=1619009317128',
           'Referer': 'https://accounts.douban.com/passport/login_popup?login_source=anony',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}
    url='https://movie.douban.com/top250?start=%d'%page_num
    time.sleep(random.randint(1,3)*2)
    response = requests.get(url,headers=headers)
    content = response.text
    html = etree.HTML(content)
    
    datas= html.xpath('//*[@id="content"]/div/div[1]/ol')
    
    for data in datas:
        rank = "".join(data.xpath('li[1]/div/div[1]/em/text()'))
        title= "".join(data.xpath('li[1]/div/div[2]/div[1]/a/span[1]/text()'))
        movie_type = "".join(data.xpath('li[1]/div/div[2]/div[1]/a/span[3]/text()'))
        star= "".join(data.xpath('li[1]/div/div[2]/div[2]/div/span[2]/text()'))
        quote = "".join(data.xpath('li[1]/div/div[2]/div[2]/p[2]/span/text()'))
        movieURL = "".join(data.xpath('li[1]/div/div[1]/a/@href'))
        imageURL= "".join(data.xpath('li[1]/div/div[1]/a/img/@src'))
        # 打开数据库连接
        conn = pymysql.connect(  # 连接本地数据库
            host="localhost",
            user="root",
            password="9834459687", # 填上自己的密码
            database="test",
            charset="utf8"
        )
        # 使用cursor()方法获取操作游标 
        cursor = conn.cursor()
        # SQL 插入语句
        sql = """
        INSERT INTO `MovieInfo`(`MovieRank`,`MovieTitle`,`MovieType`,`MovieStar`,`MovieQuote`,`MoiveURL`,`ImageURL`)
          VALUES (%s, %s, %s, %s, %s, %s, %s)
          """
        data=[(rank,title,movie_type,star,quote,movieURL,imageURL)]
        try:
            cursor.executemany(sql,data)
            conn.commit()
            print(('第'+str(rank)+'名的电影'+str(title)+'成功爬取！'))
        except Exception as e:
            print(e)
            # 发生错误则回滚
            conn.rollback()
        conn.close()

In [18]:
num = 0
while(num<250):
    crawling(num)
    num = num + 1

第1名的电影肖申克的救赎成功爬取！
第2名的电影霸王别姬成功爬取！
第3名的电影阿甘正传成功爬取！
第4名的电影这个杀手不太冷成功爬取！
第5名的电影泰坦尼克号成功爬取！
第6名的电影美丽人生成功爬取！
第7名的电影千与千寻成功爬取！
第8名的电影辛德勒的名单成功爬取！
第9名的电影盗梦空间成功爬取！
第10名的电影忠犬八公的故事成功爬取！
第11名的电影星际穿越成功爬取！
第12名的电影楚门的世界成功爬取！
第13名的电影海上钢琴师成功爬取！
第14名的电影三傻大闹宝莱坞成功爬取！
第15名的电影机器人总动员成功爬取！
第16名的电影放牛班的春天成功爬取！
第17名的电影大话西游之大圣娶亲成功爬取！
第18名的电影疯狂动物城成功爬取！
第19名的电影无间道成功爬取！
第20名的电影熔炉成功爬取！
第21名的电影教父成功爬取！
第22名的电影当幸福来敲门成功爬取！
第23名的电影龙猫成功爬取！
第24名的电影怦然心动成功爬取！
第25名的电影控方证人成功爬取！
第26名的电影触不可及成功爬取！
第27名的电影蝙蝠侠：黑暗骑士成功爬取！
第28名的电影末代皇帝成功爬取！
第29名的电影寻梦环游记成功爬取！
第30名的电影活着成功爬取！
第31名的电影何以为家成功爬取！
第32名的电影乱世佳人成功爬取！
第33名的电影哈利·波特与魔法石成功爬取！
第34名的电影指环王3：王者无敌成功爬取！
第35名的电影飞屋环游记成功爬取！
第36名的电影摔跤吧！爸爸成功爬取！
第37名的电影素媛成功爬取！
第38名的电影少年派的奇幻漂流成功爬取！
第39名的电影十二怒汉成功爬取！
第40名的电影哈尔的移动城堡成功爬取！
第41名的电影鬼子来了成功爬取！
第42名的电影我不是药神成功爬取！
第43名的电影大话西游之月光宝盒成功爬取！
第44名的电影天空之城成功爬取！
第45名的电影罗马假日成功爬取！
第46名的电影闻香识女人成功爬取！
第47名的电影辩护人成功爬取！
第48名的电影天堂电影院成功爬取！
第49名的电影猫鼠游戏成功爬取！
第50名的电影大闹天宫成功爬取！
第51名的电影钢琴家成功爬取！
第52名的电影指环王2：双塔奇兵成功爬取！
第53名的电影教父2成功爬取！
第54名的电影狮子王成功爬取！
第55名的电影搏击俱乐部成功爬取！
第56名的电影死亡诗社成功爬取！
第57名的